In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
from tqdm import tqdm
import pandas as pd
from linking import link
import json

directory = "../Labeled_Sentiment/"

posts = pd.read_csv(os.path.join(directory, "posts_with_sentiment.csv"))
comments = pd.read_csv(os.path.join(directory, "comments_with_sentiment.csv"))





In [17]:
from entity_extraction import EntityExtractor
entity_extractor = EntityExtractor()

# get the first entity in each post/comment from the entity extractor
posts["entity"] = entity_extractor.batch_extract(posts["body"],first_only=True,stats_enabled=True)
comments["entity"] = entity_extractor.batch_extract(comments["body"].astype(str),first_only=True,stats_enabled=True)

Posts with multiple entities:  896
Total posts with entities:  3376
Percent of posts with multiple entities:  26.540284360189574

Posts with multiple entities:  5589
Total posts with entities:  15071
Percent of posts with multiple entities:  37.08446685687745



In [20]:

post_ids = posts[["idstr","entity"]].values
comment_ids = comments[["parent","idstr","entity"]].values

linking = link(post_ids,comment_ids) # create dictionary with comment structure
json_file = open("linking.json", "w")
json.dump(linking, json_file, indent=4)

comments["entity"] = comment_ids[:,2] # replace entities with the linked entities
comments.to_csv('entities.csv')
comments.head()

,idstr,created,parent,submission,body,score,predicted_label,confidence,entity
0,t1_fkvzrtb,2020-03-19 01:10:44,t3_fkzxge,t3_fkzxge,This is a great idea!!! Thanks for starting this.,14.0,positive,0.989029,COMM 431
1,t1_fkvzyub,2020-03-19 01:12:43,t1_fkvypyl,t3_fkw1nj,"To me, the problem was that they didn't tell u...",4.0,negative,0.662670,NONE
2,t1_fkw04uf,2020-03-19 01:14:27,t1_fkvs07g,t3_fkw1nj,Plus there were people who may have left campu...,2.0,negative,0.575662,NONE
3,t1_fkw0fg1,2020-03-19 01:17:31,t1_fkvzyub,t3_fkw1nj,I think it was pretty clear that when they say...,8.0,neutral,0.581127,NONE
4,t1_fkw0gqm,2020-03-19 01:17:54,t1_fkw04uf,t3_fkw1nj,"Yeah that's ass for those people, I sympathize...",1.0,negative,0.880160,NONE
